In [1]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Split Data

In [2]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                             ])

# Train Model

In [3]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [4]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))

Files already downloaded
3000
750


In [5]:
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [56]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if False:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()

In [102]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(320, 160)
        self.fc2 = nn.Linear(160, 80)
        self.fc3 = nn.Linear(80, 10)

    def forward(self, x):
        #print(x.data.size())
        x = x.view(-1, 1, 28,28)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc3(x))
        return F.log_softmax(x)
classifier = Classifier()

In [113]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False


def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)



In [114]:
# CPU only training
def generate_train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

    
def generate_test(epoch, test_loader):
    model.eval()
    test_loss = 0
    for data, _ in test_loader:
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))



In [115]:
#Generative phase
for epoch in range(1, 20):
    generate_train(epoch)
    generate_test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: -1851.565186
====> Epoch: 1 Average loss: -7463.6835
====> Test set loss: -9509.6973
Train Epoch: 2 [0/3000 (0%)]	Loss: -9677.851562
====> Epoch: 2 Average loss: -9857.4584
====> Test set loss: -9889.9778
Train Epoch: 3 [0/3000 (0%)]	Loss: -10056.569336
====> Epoch: 3 Average loss: -10054.9688
====> Test set loss: -9978.5220
Train Epoch: 4 [0/3000 (0%)]	Loss: -10145.400391
====> Epoch: 4 Average loss: -10132.9395
====> Test set loss: -10031.9572
Train Epoch: 5 [0/3000 (0%)]	Loss: -10300.203125


KeyboardInterrupt: 

In [111]:
# CPU only training
def train(epoch):
    classifier.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        gen_data, mu, logvar = model(data)
        #Check reconstruction loss 
        recon_loss = reconstruction_function(gen_data, data)
        print("Reconstruction loss is", recon_loss)
        print("Type of generated data", type(gen_data))
        
        optimizer.zero_grad()
        output = classifier(gen_data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        gen_data, mu, logvar = model(data)
        output = classifier(gen_data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [112]:
#Classification phase
for epoch in range(1, 10):
    train(epoch)
    test(epoch, valid_loader)

Reconstruction loss is Variable containing:
1.00000e+05 *
 -1.5178
[torch.FloatTensor of size 1]

Type of generated data <class 'torch.autograd.variable.Variable'>
Train Epoch: 1 [0/3000 (0%)]	Loss: 2.299191
Reconstruction loss is Variable containing:
1.00000e+05 *
 -1.6905
[torch.FloatTensor of size 1]

Type of generated data <class 'torch.autograd.variable.Variable'>
Reconstruction loss is Variable containing:
1.00000e+05 *
 -2.1476
[torch.FloatTensor of size 1]

Type of generated data <class 'torch.autograd.variable.Variable'>
Reconstruction loss is Variable containing:
1.00000e+05 *
 -1.9056
[torch.FloatTensor of size 1]

Type of generated data <class 'torch.autograd.variable.Variable'>
Reconstruction loss is Variable containing:
1.00000e+05 *
 -1.8477
[torch.FloatTensor of size 1]

Type of generated data <class 'torch.autograd.variable.Variable'>
Reconstruction loss is Variable containing:
1.00000e+05 *
 -1.9292
[torch.FloatTensor of size 1]

Type of generated data <class 'torch.a

KeyboardInterrupt: 

# Create Sample Submission

In [59]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

In [60]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [61]:
test(1, test_loader)


Test set: Average loss: 0.1253, Accuracy: 9586/10000 (96%)



In [62]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [63]:
label_predict

array([ 7.,  2.,  1., ...,  4.,  5.,  6.])

In [64]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [65]:
predict_label.head()

,ID,label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4


In [68]:
predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)
